In [ ]:
#k = sqrt(2), mu=1,alpha=1, wavefunction is :
#u(x,t) = 2 sech(\sqrt(2)(x-2t))e^{i (x+t)}

In [ ]:
from __future__ import division
import os, sys, time, random
import math
import scipy
from scipy import constants
import torch
from torch import nn, optim
from torch import autograd
from torch.autograd import grad
import autograd.numpy as np
from torch.utils.data import Dataset, DataLoader
from torch.autograd.variable import Variable
from torchvision import transforms, datasets
import matplotlib.pyplot as plt
from torch.nn import functional as F
from scipy.constants import pi

In [ ]:
class Potential(nn.Module):
    def __init__(self):
        super(Potential,self).__init__()
        self.hidden0 = nn.Sequential(
            nn.Linear(2,128),
            nn.Tanh()
        )
#         self.hidden1 = nn.Sequential(
#             nn.Linear(32,128),
#             nn.Tanh()
#         )
        self.hidden1 = nn.Sequential(
            nn.Linear(128,128),
            nn.Tanh()
        )
        self.out = nn.Sequential(
            nn.Linear(128,1)
         #   nn.Sigmoid()
        )
        
    def forward(self, x):
        x = self.hidden0(x)
        x = x + self.hidden1(x)
     #   x = x + self.hidden2(x)
        x = self.out(x)
        return x

In [ ]:
def sech(x):
    return 2/(torch.exp(x)+torch.exp(-x))

def init_wave_function(x,t):
    return 2*sech(np.sqrt(2)*(x-(2*t)))*torch.cos(x+t), 2*sech(np.sqrt(2)*(x-(2*t)))*torch.sin(x+t)

In [ ]:
#imposing the Schrodinger equation 
#iu_t + u_xx + Vu =0 , 
def u_xx(batch): 
    batch.requires_grad_(True)
    x = batch[:,0]
    x.requires_grad_(True)
    t = batch[:,1]
    t.requires_grad_(True)
    output = init_wave_function(x,t)
    output_real = init_wave_function(x,t)[0]
    output_complex = init_wave_function(x,t)[1]
   # print(output,output_real,output_complex)
    output_real.requires_grad_(True)
    output_complex.requires_grad_(True)
    dHdx_real = grad(output_real, x, grad_outputs = torch.ones_like(x), 
                    create_graph=True, retain_graph=True, 
                   only_inputs=True,
                   allow_unused=True
                  )[0]
    d2Hdx2_real = grad(dHdx_real, x, grad_outputs = torch.ones_like(x), 
                    create_graph=True, retain_graph=True, 
                   only_inputs=True,
                   allow_unused=True
                  )[0]
    dHdx_complex = grad(output_complex, x, grad_outputs = torch.ones_like(x), 
                    create_graph=True, retain_graph=True, 
                   only_inputs=True,
                   allow_unused=True
                  )[0]
    d2Hdx2_complex = grad(dHdx_complex, x, grad_outputs = torch.ones_like(x), 
                    create_graph=True, retain_graph=True, 
                   only_inputs=True,
                   allow_unused=True
                  )[0]
    return d2Hdx2_real, d2Hdx2_complex
    
    
    
    

In [ ]:
def u_t(batch): 
    batch.requires_grad_(True)
    x = batch[:,0]
    x.requires_grad_(True)
    t = batch[:,1]
    t.requires_grad_(True)
    output = init_wave_function(x,t)
    output_real = init_wave_function(x,t)[0]
    output_complex = init_wave_function(x,t)[1]
   # print(output,output_real,output_complex)
    output_real.requires_grad_(True)
    output_complex.requires_grad_(True)
    dHdt_real = grad(output_real, t, grad_outputs = torch.ones_like(t), 
                    create_graph=True, retain_graph=True, 
                   only_inputs=True,
                   allow_unused=True
                  )[0]
    
    dHdt_complex = grad(output_complex, t, grad_outputs = torch.ones_like(t), 
                    create_graph=True, retain_graph=True, 
                   only_inputs=True,
                   allow_unused=True
                  )[0]
    
    return dHdt_real, dHdt_complex
    

In [ ]:
potential = Potential()
optimizer = torch.optim.Adam(potential.parameters(), lr = .0001)



#Schrodinger
#iu_t + u_xx + Vu =0 
def Schrodinger(batch):
    batch.requires_grad_(True)
    potential_energy = potential(batch)
    
    x = batch[:,0]
    x.requires_grad_(True)
    t = batch[:,1]
    t.requires_grad_(True)
    output_real = init_wave_function(x,t)[0]
    output_complex = init_wave_function(x,t)[1]
   # print(output_real,output_complex)
    real = -u_t(batch)[1] + u_xx(batch)[0] + potential_energy.squeeze()*output_real
    complex1 = u_t(batch)[0] + u_xx(batch)[1] + potential_energy.squeeze()*output_complex
   # print(real.shape,complex1.shape)
    return real**2 + complex1**2
    

In [ ]:
potential

In [ ]:
class MyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data.float()
        self.transform = transform

    def __getitem__(self, index):
        x = self.data[index]
        if self.transform:
            x = self.transform(x)
        return x

    def __len__(self):
        return len(self.data)

In [ ]:
data = torch.rand(3000,2)
dataset = MyDataset(data)
loader = DataLoader(dataset, batch_size = 32, shuffle = True)

In [ ]:
num_epochs = 1000
loss = []

for epoch in range(num_epochs):
    for n_batch, batch in enumerate(loader):
        n_data = Variable(batch, requires_grad=True)
       
       
        error = Schrodinger(n_data).mean()
 
        error.backward(retain_graph=True)
      # torch.nn.utils.clip_grad_norm_(potential.parameters(), .9)
       
        optimizer.step()
    loss.append(error)


In [ ]:
X = torch.rand(100,2)
x = X[:,0]
t = X[:,1]

In [ ]:
def ground(x,t):
    return (2*sech(np.sqrt(2)*(x-(2*t)))*torch.cos(x+t))**2 + (2*sech(np.sqrt(2)*(x-(2*t)))*torch.sin(x+t))**2

In [ ]:
torch.mean((potential(X)-ground(x,t))**2)/10